In [2]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

import matplotlib.pyplot as plt

from random import seed

# Import train_test_split function
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")

# für GPU:
# tf.device("gpu:0")

from tensorflow.keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/

In [4]:
%run '/home/jovyan/rna/_functions/functions.py'

### Load MNIST images

In [5]:
# # MNIST von sklearn mit 28x28 Pixel
# from sklearn.datasets import fetch_openml

# # Load data from https://www.openml.org/d/554
# X_raw, y_raw = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [6]:
# Import data that was saved on this system

raw_data = load_file(file = "/home/jovyan/rna/MNIST/Raw_Data/MNIST_images.pkl")
X_raw = raw_data[0]
y_raw = raw_data[1]

In [7]:
# Filter on indices of '0' and '8'
indices = (y_raw == '0') | (y_raw == '8')

# Reshape into squares
X_raw = X_raw[indices, :]
X = X_raw.reshape([sum(indices), 28, 28])

y = y_raw[indices]

# Factorize into dichotomous variable
Y = pd.factorize(y)[0]

In [8]:
# We analyse 1998 samples
X = X[0:1998]
y = y[0:1998]
Y = Y[0:1998]

np.unique(y, return_counts=True)

(array(['0', '8'], dtype=object), array([1032,  966]))

### Load Persistences Landscapes

In [9]:
pl_folderpath = "/home/jovyan/rna/MNIST/persistence_landscapes_averages/"

# Parameter for persistence landscapes
pl_resolution = 250
pl_num_landscapes = 10

In [10]:
# # Shows filenames of loaded files
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

In [11]:
avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
bucket01_H0_scaledWithin_rs = tf.reshape(avgPL_bucket01_H0_scaledWithin, [1998, pl_num_landscapes, pl_resolution, 1])
bucket01_H1_scaledWithin_rs = tf.reshape(avgPL_bucket01_H1_scaledWithin, [1998, pl_num_landscapes, pl_resolution, 1])

X_b01_scaledWithin_cnn = tf.concat(axis=3, values = [bucket01_H0_scaledWithin_rs, bucket01_H1_scaledWithin_rs]).numpy()


avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
bucket01_H0_scaledBetween_rs = tf.reshape(avgPL_bucket01_H0_scaledBetween, [1998, pl_num_landscapes, pl_resolution, 1])
bucket01_H1_scaledBetween_rs = tf.reshape(avgPL_bucket01_H1_scaledBetween, [1998, pl_num_landscapes, pl_resolution, 1])

X_b01_scaledBetween_cnn = tf.concat(axis=3, values = [bucket01_H0_scaledBetween_rs, bucket01_H1_scaledBetween_rs]).numpy()


avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])
bucket01_H0_unscaled_rs = tf.reshape(avgPL_bucket01_H0_unscaled, [1998, pl_num_landscapes, pl_resolution, 1])
bucket01_H1_unscaled_rs = tf.reshape(avgPL_bucket01_H1_unscaled, [1998, pl_num_landscapes, pl_resolution, 1])

X_b01_unscaled_cnn = tf.concat(axis=3, values = [bucket01_H0_unscaled_rs, bucket01_H1_unscaled_rs]).numpy()

### Train-Test-Splits

In [12]:
seed(999)
X_cnn_b01_scaledBetween_train, X_cnn_b01_scaledBetween_test, y_cnn_b01_scaledBetween_train, y_cnn_b01_scaledBetween_test = train_test_split(X_b01_scaledBetween_cnn, 
                                                                                                                                            Y, 
                                                                                                                                            test_size = 0.2)

seed(999)
X_cnn_b01_scaledWithin_train, X_cnn_b01_scaledWithin_test, y_cnn_b01_scaledWithin_train, y_cnn_b01_scaledWithin_test = train_test_split(X_b01_scaledWithin_cnn,
                                                                                                                                        Y, 
                                                                                                                                        test_size = 0.2)

seed(999)
X_cnn_b01_unscaled_train, X_cnn_b01_unscaled_test, y_cnn_b01_unscaled_train, y_cnn_b01_unscaled_test = train_test_split(X_b01_unscaled_cnn, 
                                                                                                                        Y, 
                                                                                                                        test_size = 0.2)

In [13]:
# Rawdata
X_rs = tf.reshape(X, [1998, 28, 28, 1])

# Train-Test-Split raw images
seed(999)
X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X_rs.numpy(),
                                                                    Y,
                                                                    test_size = 0.3)

# Convolutional Neural Networks on Raw Data

In [14]:
no_relevant_samples = len(avgPL_bucket01_H1_unscaled)
no_relevant_samples

1998

### CNN Models for raw data

In [15]:
# Folderpath to store the results
folderpath_results = "/home/jovyan/rna/MNIST/results/"

In [16]:
def CNN_model_paper_raw():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(28, 28, 1)
                       )
                )
    model_cnn.add(layers.MaxPool2D((2,2)))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2, 2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(32, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='softmax'))
        
    return model_cnn

def CNN_model_paper2_raw():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(28, 28, 1),
                        padding='same'
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(32, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='sigmoid'))
        
    return model_cnn

def CNN_model_paper_withDropout_raw():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(28, 28, 1),
                        padding='same'
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Dropout(0.25))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='softmax'))
        
    return model_cnn

def simple_CNN_model_1_raw():
    
    model_cnn = Sequential()
    
    model_cnn.add(Conv2D(32,
                         kernel_size=3, 
                         activation='elu', 
                         input_shape=(28, 28, 1),
                         padding='same'
                        )
                 )
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(1, activation='softmax'))
    
    return model_cnn

def simple_CNN_model_1_sigmoid_raw():
    
    model_cnn = Sequential()
    
    model_cnn.add(Conv2D(32,
                         kernel_size=3, 
                         activation='elu', 
                         input_shape=(28, 28, 1),
                         padding='same'
                        )
                 )
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(1, activation='sigmoid'))
    
    return model_cnn

def cnn_model_succeeded_raw():

    model = Sequential()
    
    # add model layers
    model.add(Conv2D(64,
                     kernel_size=3,
                     activation='elu',
                     input_shape=(28, 28, 1),
                     padding='same'
                    )
             )
    
    model.add(layers.MaxPool2D((2)))
    model.add(Flatten())
    model.add(Dense(32, activation='elu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [17]:
# Hyperparameters for gridsearch
cnn_raw_models = [CNN_model_paper_raw, CNN_model_paper2_raw, simple_CNN_model_1_raw, simple_CNN_model_1_sigmoid_raw, cnn_model_succeeded_raw]
cnn_raw_learning_rates = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
cnn_raw_validation_splits = [0.33, None]
cnn_raw_epochs = [1, 3, 5, 10, 20, 50, 100]

In [18]:
grid_cnn_raw = create_modelgrid(models = cnn_raw_models,
                                learning_rates = cnn_raw_learning_rates,
                                validation_splits = cnn_raw_validation_splits,
                                epochs = cnn_raw_epochs)

In [19]:
results_cnn_raw = test_multiple_models(x_train = X_raw_train,
                                       y_train = y_raw_train,
                                       x_test = X_raw_test,
                                       y_test = y_raw_test,
                                       modelgrid = grid_cnn_raw)

Training model 1 of 350


2022-09-07 08:37:17.068014: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Training model 2 of 350
Training model 3 of 350
Training model 4 of 350
Training model 5 of 350
Training model 6 of 350
Training model 7 of 350
Training model 8 of 350
Training model 9 of 350
Training model 10 of 350
Training model 11 of 350
Training model 12 of 350
Training model 13 of 350
Training model 14 of 350
Training model 15 of 350
Training model 16 of 350
Training model 17 of 350
Training model 18 of 350
Training model 19 of 350
Training model 20 of 350
Training model 21 of 350
Training model 22 of 350
Training model 23 of 350
Training model 24 of 350
Training model 25 of 350
Training model 26 of 350
Training model 27 of 350
Training model 28 of 350
Training model 29 of 350
Training model 30 of 350
Training model 31 of 350
Training model 32 of 350
Training model 33 of 350
Training model 34 of 350
Training model 35 of 350
Training model 36 of 350
Training model 37 of 350
Training model 38 of 350
Training model 39 of 350
Training model 40 of 350
Training model 41 of 350
Training

Training model 322 of 350
Training model 323 of 350
Training model 324 of 350
Training model 325 of 350
Training model 326 of 350
Training model 327 of 350
Training model 328 of 350
Training model 329 of 350
Training model 330 of 350
Training model 331 of 350
Training model 332 of 350
Training model 333 of 350
Training model 334 of 350
Training model 335 of 350
Training model 336 of 350
Training model 337 of 350
Training model 338 of 350
Training model 339 of 350
Training model 340 of 350
Training model 341 of 350
Training model 342 of 350
Training model 343 of 350
Training model 344 of 350
Training model 345 of 350
Training model 346 of 350
Training model 347 of 350
Training model 348 of 350
Training model 349 of 350
Training model 350 of 350


In [20]:
results_cnn_raw = results_cnn_raw.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_raw.to_csv(folderpath_results + "results_simple_CNN_rawdata.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_raw.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,CNN_model_paper2_raw,0.0010,0.33,20,1.000000,1.000000,1.000000
1,simple_CNN_model_1_sigmoid_raw,0.0001,0.33,20,0.998333,0.996416,1.000000
2,CNN_model_paper2_raw,0.0010,0.33,100,0.998333,0.996416,1.000000
3,simple_CNN_model_1_sigmoid_raw,0.0001,0.33,5,0.998333,0.996416,1.000000
4,cnn_model_succeeded_raw,0.0010,0.33,10,0.998333,0.996416,1.000000
5,cnn_model_succeeded_raw,0.0010,None,100,0.998333,0.996416,1.000000
6,cnn_model_succeeded_raw,0.0001,0.33,50,0.998333,0.996416,1.000000
7,CNN_model_paper2_raw,0.0001,None,50,0.996667,0.992832,1.000000
8,CNN_model_paper2_raw,0.0010,0.33,5,0.996667,0.992832,1.000000
9,simple_CNN_model_1_sigmoid_raw,0.0001,None,5,0.996667,1.000000,0.993769


# Convolutional Neural Networks on Persistence Landscapes

In [21]:
def CNN_model_paper():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(10, 250, 2)
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2, 2)))
    
#     model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
#     model_cnn.add(layers.MaxPool2D((2, 2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(32, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='softmax'))
        
    return model_cnn

def CNN_model_paper2():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(10, 250, 2),
                        padding='same'
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(32, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='sigmoid'))
        
    return model_cnn

def CNN_model_paper_withDropout():
    
    model_cnn = Sequential()

    model_cnn.add(Conv2D(64,
                        kernel_size=3, 
                        activation='elu', 
                        input_shape=(10, 250, 2),
                        padding='same'
                       )
                )
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Dropout(0.25))
    
    model_cnn.add(Conv2D(64, kernel_size=3, activation='elu'))
    model_cnn.add(layers.MaxPool2D((2)))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    model_cnn.add(Dense(1, activation='softmax'))
        
    return model_cnn

def simple_CNN_model_1():
    
    model_cnn = Sequential()
    
    model_cnn.add(Conv2D(32,
                         kernel_size=3, 
                         activation='elu', 
                         input_shape=(10, 250, 2),
                         padding='same'
                        )
                 )
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(1, activation='softmax'))
    
    return model_cnn

def simple_CNN_model_1_sigmoid():
    
    model_cnn = Sequential()
    
    model_cnn.add(Conv2D(32,
                         kernel_size=3, 
                         activation='elu', 
                         input_shape=(10, 250, 2),
                         padding='same'
                        )
                 )
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(16, activation='elu'))
    
    model_cnn.add(Flatten())
    
    model_cnn.add(Dense(1, activation='sigmoid'))
    
    return model_cnn

def cnn_model_succeeded():

    model = Sequential()
    
    # add model layers
    model.add(Conv2D(64,
                     kernel_size=3,
                     activation='elu',
                     input_shape=(10, 250, 2),
                     padding='same'
                    )
             )
    
    model.add(layers.MaxPool2D((2)))
    model.add(Flatten())
    model.add(Dense(32, activation='elu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [22]:
cnn_models = [CNN_model_paper, CNN_model_paper2, CNN_model_paper_withDropout, simple_CNN_model_1, simple_CNN_model_1_sigmoid, cnn_model_succeeded]
cnn_learning_rates = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
cnn_validation_splits = [0.33, None]
cnn_epochs = [1, 3, 5, 10, 20, 50, 100]

### Train CNN-Models on Persistence Landscapes (H1 & H0; scaledBetween)

In [23]:
grid_cnn_scaledBetween = create_modelgrid(models = cnn_models,
                                          learning_rates = cnn_learning_rates,
                                          validation_splits = cnn_validation_splits,
                                          epochs = cnn_epochs)

In [24]:
results_cnn_b01_scaledBetween = test_multiple_models(x_train = X_cnn_b01_scaledBetween_train,
                                                     y_train = y_cnn_b01_scaledBetween_train,
                                                     x_test = X_cnn_b01_scaledBetween_test,
                                                     y_test = y_cnn_b01_scaledBetween_test,
                                                     modelgrid = grid_cnn_scaledBetween)

Training model 1 of 420
Training model 2 of 420
Training model 3 of 420
Training model 4 of 420
Training model 5 of 420
Training model 6 of 420
Training model 7 of 420
Training model 8 of 420
Training model 9 of 420
Training model 10 of 420
Training model 11 of 420
Training model 12 of 420
Training model 13 of 420
Training model 14 of 420
Training model 15 of 420
Training model 16 of 420
Training model 17 of 420
Training model 18 of 420
Training model 19 of 420
Training model 20 of 420
Training model 21 of 420
Training model 22 of 420
Training model 23 of 420
Training model 24 of 420
Training model 25 of 420
Training model 26 of 420
Training model 27 of 420
Training model 28 of 420
Training model 29 of 420
Training model 30 of 420
Training model 31 of 420
Training model 32 of 420
Training model 33 of 420
Training model 34 of 420
Training model 35 of 420
Training model 36 of 420
Training model 37 of 420
Training model 38 of 420
Training model 39 of 420
Training model 40 of 420
Training 

Training model 321 of 420
Training model 322 of 420
Training model 323 of 420
Training model 324 of 420
Training model 325 of 420
Training model 326 of 420
Training model 327 of 420
Training model 328 of 420
Training model 329 of 420
Training model 330 of 420
Training model 331 of 420
Training model 332 of 420
Training model 333 of 420
Training model 334 of 420
Training model 335 of 420
Training model 336 of 420
Training model 337 of 420
Training model 338 of 420
Training model 339 of 420
Training model 340 of 420
Training model 341 of 420
Training model 342 of 420
Training model 343 of 420
Training model 344 of 420
Training model 345 of 420
Training model 346 of 420
Training model 347 of 420
Training model 348 of 420
Training model 349 of 420
Training model 350 of 420
Training model 351 of 420
Training model 352 of 420
Training model 353 of 420
Training model 354 of 420
Training model 355 of 420
Training model 356 of 420
Training model 357 of 420
Training model 358 of 420
Training mod

In [25]:
results_cnn_b01_scaledBetween = results_cnn_b01_scaledBetween.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_b01_scaledBetween.to_csv(folderpath_results + "results_simple_CNN_b01_scaledBetween_H_0_1.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_b01_scaledBetween.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_CNN_model_1_sigmoid,1.000000e-06,None,100,0.5825,0.560976,0.605128
1,cnn_model_succeeded,1.000000e-05,0.33,20,0.5750,0.804878,0.333333
2,simple_CNN_model_1_sigmoid,1.000000e-05,None,100,0.5750,0.443902,0.712821
3,simple_CNN_model_1_sigmoid,1.000000e-07,None,50,0.5675,0.565854,0.569231
4,simple_CNN_model_1_sigmoid,1.000000e-06,None,3,0.5650,0.624390,0.502564
5,CNN_model_paper2,1.000000e-03,None,20,0.5625,0.497561,0.630769
6,cnn_model_succeeded,1.000000e-06,0.33,10,0.5575,0.560976,0.553846
7,CNN_model_paper2,1.000000e-03,None,5,0.5525,0.317073,0.800000
8,cnn_model_succeeded,1.000000e-06,0.33,20,0.5525,0.697561,0.400000
9,cnn_model_succeeded,1.000000e-07,0.33,50,0.5500,0.482927,0.620513


### Train CNN-Models on Persistence Landscapes (H1 & H0; scaledWithin)

In [26]:
grid_cnn_scaledWithin = create_modelgrid(models = cnn_models,
                                         learning_rates = cnn_learning_rates,
                                         validation_splits = cnn_validation_splits,
                                         epochs = cnn_epochs)

In [27]:
results_cnn_b01_scaledWithin = test_multiple_models(x_train = X_cnn_b01_scaledWithin_train,
                                                    y_train = y_cnn_b01_scaledWithin_train,
                                                    x_test = X_cnn_b01_scaledWithin_test,
                                                    y_test = y_cnn_b01_scaledWithin_test,
                                                    modelgrid = grid_cnn_scaledWithin)

Training model 1 of 420
Training model 2 of 420
Training model 3 of 420
Training model 4 of 420
Training model 5 of 420
Training model 6 of 420
Training model 7 of 420
Training model 8 of 420
Training model 9 of 420
Training model 10 of 420
Training model 11 of 420
Training model 12 of 420
Training model 13 of 420
Training model 14 of 420
Training model 15 of 420
Training model 16 of 420
Training model 17 of 420
Training model 18 of 420
Training model 19 of 420
Training model 20 of 420
Training model 21 of 420
Training model 22 of 420
Training model 23 of 420
Training model 24 of 420
Training model 25 of 420
Training model 26 of 420
Training model 27 of 420
Training model 28 of 420
Training model 29 of 420
Training model 30 of 420
Training model 31 of 420
Training model 32 of 420
Training model 33 of 420
Training model 34 of 420
Training model 35 of 420
Training model 36 of 420
Training model 37 of 420
Training model 38 of 420
Training model 39 of 420
Training model 40 of 420
Training 

Training model 321 of 420
Training model 322 of 420
Training model 323 of 420
Training model 324 of 420
Training model 325 of 420
Training model 326 of 420
Training model 327 of 420
Training model 328 of 420
Training model 329 of 420
Training model 330 of 420
Training model 331 of 420
Training model 332 of 420
Training model 333 of 420
Training model 334 of 420
Training model 335 of 420
Training model 336 of 420
Training model 337 of 420
Training model 338 of 420
Training model 339 of 420
Training model 340 of 420
Training model 341 of 420
Training model 342 of 420
Training model 343 of 420
Training model 344 of 420
Training model 345 of 420
Training model 346 of 420
Training model 347 of 420
Training model 348 of 420
Training model 349 of 420
Training model 350 of 420
Training model 351 of 420
Training model 352 of 420
Training model 353 of 420
Training model 354 of 420
Training model 355 of 420
Training model 356 of 420
Training model 357 of 420
Training model 358 of 420
Training mod

In [28]:
results_cnn_b01_scaledWithin = results_cnn_b01_scaledWithin.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_b01_scaledWithin.to_csv(folderpath_results + "results_simple_CNN_b01_scaledWithin_H_0_1.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_b01_scaledWithin.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,CNN_model_paper2,1.000000e-03,None,20,0.5725,0.666667,0.485577
1,cnn_model_succeeded,1.000000e-07,None,100,0.5625,0.562500,0.562500
2,CNN_model_paper2,1.000000e-04,None,100,0.5575,0.734375,0.394231
3,simple_CNN_model_1_sigmoid,1.000000e-04,None,3,0.5575,0.427083,0.677885
4,CNN_model_paper2,1.000000e-04,None,20,0.5550,0.078125,0.995192
5,cnn_model_succeeded,1.000000e-06,None,100,0.5500,0.458333,0.634615
6,cnn_model_succeeded,1.000000e-05,0.33,1,0.5500,0.401042,0.687500
7,simple_CNN_model_1_sigmoid,1.000000e-04,None,20,0.5500,0.203125,0.870192
8,cnn_model_succeeded,1.000000e-06,0.33,20,0.5500,0.463542,0.629808
9,simple_CNN_model_1_sigmoid,1.000000e-07,0.33,5,0.5475,0.317708,0.759615


### Train CNN-Models on Persistence Landscapes (H1 & H0; unscaled)

In [29]:
grid_cnn_unscaled = create_modelgrid(models = cnn_models,
                                     learning_rates = cnn_learning_rates,
                                     validation_splits = cnn_validation_splits,
                                     epochs = cnn_epochs)

In [30]:
results_cnn_b01_unscaled = test_multiple_models(x_train = X_cnn_b01_unscaled_train,
                                                y_train = y_cnn_b01_unscaled_train,
                                                x_test = X_cnn_b01_unscaled_test,
                                                y_test = y_cnn_b01_unscaled_test,
                                                modelgrid = grid_cnn_unscaled)

Training model 1 of 420
Training model 2 of 420
Training model 3 of 420
Training model 4 of 420
Training model 5 of 420
Training model 6 of 420
Training model 7 of 420
Training model 8 of 420
Training model 9 of 420
Training model 10 of 420
Training model 11 of 420
Training model 12 of 420
Training model 13 of 420
Training model 14 of 420
Training model 15 of 420
Training model 16 of 420
Training model 17 of 420
Training model 18 of 420
Training model 19 of 420
Training model 20 of 420
Training model 21 of 420
Training model 22 of 420
Training model 23 of 420
Training model 24 of 420
Training model 25 of 420
Training model 26 of 420
Training model 27 of 420
Training model 28 of 420
Training model 29 of 420
Training model 30 of 420
Training model 31 of 420
Training model 32 of 420
Training model 33 of 420
Training model 34 of 420
Training model 35 of 420
Training model 36 of 420
Training model 37 of 420
Training model 38 of 420
Training model 39 of 420
Training model 40 of 420
Training 

Training model 321 of 420
Training model 322 of 420
Training model 323 of 420
Training model 324 of 420
Training model 325 of 420
Training model 326 of 420
Training model 327 of 420
Training model 328 of 420
Training model 329 of 420
Training model 330 of 420
Training model 331 of 420
Training model 332 of 420
Training model 333 of 420
Training model 334 of 420
Training model 335 of 420
Training model 336 of 420
Training model 337 of 420
Training model 338 of 420
Training model 339 of 420
Training model 340 of 420
Training model 341 of 420
Training model 342 of 420
Training model 343 of 420
Training model 344 of 420
Training model 345 of 420
Training model 346 of 420
Training model 347 of 420
Training model 348 of 420
Training model 349 of 420
Training model 350 of 420
Training model 351 of 420
Training model 352 of 420
Training model 353 of 420
Training model 354 of 420
Training model 355 of 420
Training model 356 of 420
Training model 357 of 420
Training model 358 of 420
Training mod

In [31]:
results_cnn_b01_unscaled = results_cnn_b01_unscaled.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_cnn_b01_unscaled.to_csv(folderpath_results + "results_simple_CNN_b01_unscaled_H_0_1.csv",
                                              encoding='utf-8',
                                              index=False)

results_cnn_b01_unscaled.head(20)

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_CNN_model_1_sigmoid,1.000000e-05,0.33,10,0.5800,0.256545,0.875598
1,CNN_model_paper2,1.000000e-03,None,50,0.5775,0.539267,0.612440
2,cnn_model_succeeded,1.000000e-05,None,100,0.5625,0.617801,0.511962
3,simple_CNN_model_1_sigmoid,1.000000e-05,None,50,0.5625,0.450262,0.665072
4,simple_CNN_model_1_sigmoid,1.000000e-07,None,10,0.5500,0.565445,0.535885
5,simple_CNN_model_1_sigmoid,1.000000e-05,None,3,0.5500,0.125654,0.937799
6,cnn_model_succeeded,1.000000e-04,0.33,100,0.5475,0.366492,0.712919
7,CNN_model_paper2,1.000000e-04,None,20,0.5475,0.842932,0.277512
8,CNN_model_paper2,1.000000e-07,0.33,5,0.5475,0.623037,0.478469
9,CNN_model_paper2,1.000000e-06,0.33,1,0.5450,0.717277,0.387560
